In [1]:
import os
import json
import h5py
import numpy as np

import robomimic
import robomimic.utils.file_utils as FileUtils
import robomimic.utils.env_utils as EnvUtils
import robomimic.utils.obs_utils as ObsUtils
import imageio
import tqdm
from robomimic.utils.file_utils import create_hdf5_filter_key

/home/ns1254/miniforge3/envs/robodiff/lib/python3.9/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: libtorch_cuda_cu.so: cannot open shared object file: No such file or directory
  warn(f"Failed to load image Python extension: {e}")


In [2]:
dataset_path = "/home/ns1254/dataset_mimicgen/kitchen82_31.hdf5"

f_org = h5py.File(dataset_path, "r")
demos = list(f_org["data"].keys())

In [3]:
lengths=[]
for demo_name in demos:
    demo=f_org['data'][demo_name]
    num_samples=demo.attrs['num_samples']
    lengths.append(num_samples)

lengths=np.array(lengths)

print('Number of demos: ', len(demos))
print('Max length: ', np.max(lengths))
print('Min length: ', np.min(lengths))
print('Mean length: ', np.mean(lengths))

Number of demos:  113
Max length:  2000
Min length:  773
Mean length:  1116.0619469026549


In [4]:
f_org['mask'].keys()

<KeysViewHDF5 ['b21', 'bad', 'g30', 'g30b10', 'g30b20', 'g40', 'g40b10', 'g40b20', 'g60', 'g60b20', 'g64', 'good']>

In [5]:
def copy_group(src_group, dest_group):
    # Copy attributes
    for attr_name, attr_value in src_group.attrs.items():
        dest_group.attrs[attr_name] = attr_value

    for key, item in src_group.items():
        if isinstance(item, h5py.Group):
            # Create a new group in the destination and recursively copy contents
            new_group = dest_group.create_group(key)
            copy_group(item, new_group)
        elif isinstance(item, h5py.Dataset):
            # Copy datasets
            dataset = dest_group.create_dataset(key, data=item[...])
            # Copy attributes for the dataset
            for attr_name, attr_value in item.attrs.items():
                dataset.attrs[attr_name] = attr_value


def copy_demos(f_src, f_dest, demos2copy, next_id):
    new_demo_names=[]
    i=next_id
    for demo_name in tqdm.tqdm(demos2copy):
        demo=f_src['data'][demo_name]  

        demo_name_new="demo_"+str(i) 
        new_demo=f_dest["data"].create_group(demo_name_new) 
        copy_group(demo, new_demo) 

        new_demo_names.append(demo_name_new)
        i = i+1

    return i , new_demo_names

In [6]:
demos[:3]

['demo_1', 'demo_10', 'demo_100']

In [7]:
# dataset_path = "/home/ns1254/dataset_mimicgen/mug124.hdf5"

In [8]:
demos2copy =demos
demos2copy=sorted(demos2copy, key=lambda x: int(x.split('_')[-1]))
len(demos2copy), demos2copy

(113,
 ['demo_1',
  'demo_2',
  'demo_3',
  'demo_4',
  'demo_5',
  'demo_6',
  'demo_7',
  'demo_8',
  'demo_9',
  'demo_10',
  'demo_11',
  'demo_12',
  'demo_13',
  'demo_14',
  'demo_15',
  'demo_16',
  'demo_17',
  'demo_18',
  'demo_19',
  'demo_20',
  'demo_21',
  'demo_22',
  'demo_23',
  'demo_24',
  'demo_25',
  'demo_26',
  'demo_27',
  'demo_28',
  'demo_29',
  'demo_30',
  'demo_31',
  'demo_32',
  'demo_33',
  'demo_34',
  'demo_35',
  'demo_36',
  'demo_37',
  'demo_38',
  'demo_39',
  'demo_40',
  'demo_41',
  'demo_42',
  'demo_43',
  'demo_44',
  'demo_45',
  'demo_46',
  'demo_47',
  'demo_48',
  'demo_49',
  'demo_50',
  'demo_51',
  'demo_52',
  'demo_53',
  'demo_54',
  'demo_55',
  'demo_56',
  'demo_57',
  'demo_58',
  'demo_59',
  'demo_60',
  'demo_61',
  'demo_62',
  'demo_63',
  'demo_64',
  'demo_65',
  'demo_66',
  'demo_67',
  'demo_68',
  'demo_69',
  'demo_70',
  'demo_71',
  'demo_72',
  'demo_73',
  'demo_74',
  'demo_75',
  'demo_76',
  'demo_77',
  

In [9]:
dataset_path_sub =dataset_path
dataset_path_sub=dataset_path_sub.replace(".hdf5", "")+"_0ind.hdf5"
dataset_path_sub

'/home/ns1254/dataset_mimicgen/kitchen82_31_0ind.hdf5'

In [10]:
f_sub = h5py.File(dataset_path_sub, "w") 
f_sub.create_group("data")
f_sub.create_group("mask")

<HDF5 group "/mask" (0 members)>

In [11]:
for key in  f_org['data'].attrs.keys():
    f_sub['data'].attrs[key] = f_org['data'].attrs[key]

In [12]:
print('copying demos...') 
next_id , new_demo_names=copy_demos(f_src=f_org, f_dest=f_sub, demos2copy=demos2copy, next_id=0)

copying demos...


100%|██████████| 113/113 [01:38<00:00,  1.14it/s]


In [13]:
map={old_name:new_name for old_name,new_name in zip(demos2copy, new_demo_names)}
len(map)

113

In [22]:
# map

In [14]:
for key in f_org['mask'].keys():
    demo_names = [b.decode('utf-8') for b in f_org['mask'][key] ]
    demo_names_new = [map[demo_name] for demo_name in demo_names]

    print(f"creating filter key: {key} with {len(demo_names_new)} demos")
    demos= np.array(demo_names_new, dtype='S8')   
    filter_keys=sorted([elem for elem in demos]) 
    filter_lengths = create_hdf5_filter_key(hdf5_path=dataset_path_sub, demo_keys=filter_keys, key_name=key)

creating filter key: b21 with 21 demos
creating filter key: bad with 31 demos
creating filter key: g30 with 30 demos
creating filter key: g30b10 with 40 demos
creating filter key: g30b20 with 50 demos
creating filter key: g40 with 40 demos
creating filter key: g40b10 with 50 demos
creating filter key: g40b20 with 60 demos
creating filter key: g60 with 60 demos
creating filter key: g60b20 with 80 demos
creating filter key: g64 with 64 demos
creating filter key: good with 82 demos


In [15]:
f_sub['mask'].keys()

<KeysViewHDF5 ['b21', 'bad', 'g30', 'g30b10', 'g30b20', 'g40', 'g40b10', 'g40b20', 'g60', 'g60b20', 'g64', 'good']>

In [17]:
f_org['mask']['b21'][:]

array([b'demo_100', b'demo_101', b'demo_103', b'demo_104', b'demo_105',
       b'demo_106', b'demo_107', b'demo_108', b'demo_111', b'demo_112',
       b'demo_113', b'demo_83', b'demo_84', b'demo_85', b'demo_89',
       b'demo_90', b'demo_94', b'demo_95', b'demo_96', b'demo_98',
       b'demo_99'], dtype='|S8')

In [18]:
f_sub['mask']['b21'][:]

array([b'demo_100', b'demo_102', b'demo_103', b'demo_104', b'demo_105',
       b'demo_106', b'demo_107', b'demo_110', b'demo_111', b'demo_112',
       b'demo_82', b'demo_83', b'demo_84', b'demo_88', b'demo_89',
       b'demo_93', b'demo_94', b'demo_95', b'demo_97', b'demo_98',
       b'demo_99'], dtype='|S8')

In [19]:
f_org['data'].keys()

<KeysViewHDF5 ['demo_1', 'demo_10', 'demo_100', 'demo_101', 'demo_102', 'demo_103', 'demo_104', 'demo_105', 'demo_106', 'demo_107', 'demo_108', 'demo_109', 'demo_11', 'demo_110', 'demo_111', 'demo_112', 'demo_113', 'demo_12', 'demo_13', 'demo_14', 'demo_15', 'demo_16', 'demo_17', 'demo_18', 'demo_19', 'demo_2', 'demo_20', 'demo_21', 'demo_22', 'demo_23', 'demo_24', 'demo_25', 'demo_26', 'demo_27', 'demo_28', 'demo_29', 'demo_3', 'demo_30', 'demo_31', 'demo_32', 'demo_33', 'demo_34', 'demo_35', 'demo_36', 'demo_37', 'demo_38', 'demo_39', 'demo_4', 'demo_40', 'demo_41', 'demo_42', 'demo_43', 'demo_44', 'demo_45', 'demo_46', 'demo_47', 'demo_48', 'demo_49', 'demo_5', 'demo_50', 'demo_51', 'demo_52', 'demo_53', 'demo_54', 'demo_55', 'demo_56', 'demo_57', 'demo_58', 'demo_59', 'demo_6', 'demo_60', 'demo_61', 'demo_62', 'demo_63', 'demo_64', 'demo_65', 'demo_66', 'demo_67', 'demo_68', 'demo_69', 'demo_7', 'demo_70', 'demo_71', 'demo_72', 'demo_73', 'demo_74', 'demo_75', 'demo_76', 'demo_77',

In [20]:
f_sub['data'].keys()

<KeysViewHDF5 ['demo_0', 'demo_1', 'demo_10', 'demo_100', 'demo_101', 'demo_102', 'demo_103', 'demo_104', 'demo_105', 'demo_106', 'demo_107', 'demo_108', 'demo_109', 'demo_11', 'demo_110', 'demo_111', 'demo_112', 'demo_12', 'demo_13', 'demo_14', 'demo_15', 'demo_16', 'demo_17', 'demo_18', 'demo_19', 'demo_2', 'demo_20', 'demo_21', 'demo_22', 'demo_23', 'demo_24', 'demo_25', 'demo_26', 'demo_27', 'demo_28', 'demo_29', 'demo_3', 'demo_30', 'demo_31', 'demo_32', 'demo_33', 'demo_34', 'demo_35', 'demo_36', 'demo_37', 'demo_38', 'demo_39', 'demo_4', 'demo_40', 'demo_41', 'demo_42', 'demo_43', 'demo_44', 'demo_45', 'demo_46', 'demo_47', 'demo_48', 'demo_49', 'demo_5', 'demo_50', 'demo_51', 'demo_52', 'demo_53', 'demo_54', 'demo_55', 'demo_56', 'demo_57', 'demo_58', 'demo_59', 'demo_6', 'demo_60', 'demo_61', 'demo_62', 'demo_63', 'demo_64', 'demo_65', 'demo_66', 'demo_67', 'demo_68', 'demo_69', 'demo_7', 'demo_70', 'demo_71', 'demo_72', 'demo_73', 'demo_74', 'demo_75', 'demo_76', 'demo_77', '

In [23]:
f_org.close()
f_sub.close()

### verify

In [5]:
dataset_path = '/home/ns1254/dataset_mimicgen/mug124_0ind.hdf5'

# dataset_path = "/home/ns1254/mimicgen/datasets/source/mug_cleanup.hdf5"

f_sub = h5py.File(dataset_path, "r+")
demos = list(f_sub["data"].keys())

lengths=[]
for demo_name in demos:
    demo=f_sub['data'][demo_name]
    num_samples=demo.attrs['num_samples']
    lengths.append(num_samples)

lengths=np.array(lengths)

print('Number of demos: ', len(demos))
print('Max length: ', np.max(lengths))
print('Min length: ', np.min(lengths))
print('Mean length: ', np.mean(lengths))

f_sub['mask'].keys() 

Number of demos:  124
Max length:  1500
Min length:  465
Mean length:  818.2903225806451


<KeysViewHDF5 ['bad_first', 'bad_second', 'bad_st', 'bad_third', 'clean40', 'clean40+selected20bad', 'g20', 'g20b30', 'g30', 'g30b30', 'g40', 'g40b20', 'g40b30', 'g40b40', 'g40b50', 'g40f10', 'g40s10', 'g40st10', 'g40t10', 'g50', 'g50f10', 'g50s10', 'g50st10', 'g50t10', 'g60b30', 'g60b40', 'good', 'selected20bad']>

In [7]:
f_org['data'].attrs.keys()

<KeysViewHDF5 ['env_args']>

In [10]:
f_sub['data'].attrs.keys()

<KeysViewHDF5 ['env_args']>

In [9]:
# f_sub['data'].attrs['env_args'] = f_org['data'].attrs['env_args']

In [11]:
f_sub.close()